In [2]:
import pandas as pd
import classifiers
import importlib
from igraph import Graph
import igraph as ig
import numpy as np
from ast import literal_eval
import concurrent.futures
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
reactome_hypergeometric = pd.read_csv("../../data/processed/metrics/neglogpvalue_reactome.csv", sep=',', header=0, index_col=0)
reactome_closeness = pd.read_csv("../../data/processed/metrics/process_closeness.csv", sep=',', header=0, index_col=0)
reactome_betweenness = pd.read_csv("../../data/processed/metrics/process_betweenness.csv", sep=',', header=0, index_col=0)
reactome_rwr = pd.read_csv("../../data/processed/metrics/process_rwr.csv", sep=',', header=0, index_col=0)

In [10]:
graph = Graph.Read_GML("../../data/processed/graph")
reactome_proteins_indexes_df = pd.read_csv("../../data/processed/reactome_proteins_indexes.csv", sep=',', header=0)
disgenet_proteins_indexes_df = pd.read_csv("../../data/processed/disgenet_proteins_indexes.csv", sep=',', header=0)
reactome_neglogp_ar = np.array(reactome_hypergeometric)
disgenet_neglogp = pd.read_csv('../../data/processed/metrics/neglogpvalue_disgenet.csv', index_col=0)
disgenet_neglogp_ar = np.array(disgenet_neglogp)

In [11]:
reactome_proteins_indexes_df['protein_index'] = reactome_proteins_indexes_df['protein_index'].apply(literal_eval)
disgenet_proteins_indexes_df['protein_index'] = disgenet_proteins_indexes_df['protein_index'].apply(literal_eval)
reactome_proteins_indexes_df['proteins_ids'] = reactome_proteins_indexes_df['proteins_ids'].apply(literal_eval)
disgenet_proteins_indexes_df['proteins_ids'] = disgenet_proteins_indexes_df['proteins_ids'].apply(literal_eval)

In [13]:
reactome_labels = np.zeros(reactome_neglogp_ar.shape)
column = 0
for indexes in reactome_proteins_indexes_df['protein_index'].values:
    for pos in indexes:
        reactome_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/reactome_labels.csv", reactome_labels, delimiter=",")

In [14]:
disgenet_labels = np.zeros(disgenet_neglogp_ar.shape)
column = 0
for indexes in disgenet_proteins_indexes_df['protein_index'].values:
    for pos in indexes:
        disgenet_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/disgenet_labels.csv", disgenet_labels, delimiter=",")

In [9]:
reactome_labels_df = pd.read_csv("../../data/processed/reactome_labels.csv", sep=',', names=reactome_proteins_indexes_df['process'].values)
reactome_labels_df['protein_id'] = graph.vs['name']
reactome_labels_df.set_index('protein_id', inplace=True)

In [21]:
ppi80_hyper = pd.read_csv("../../data/processed/metrics/process_ppi80_hyper.csv", sep=',', header=0, index_col=0)
ppi80_closeness = pd.read_csv("../../data/processed/metrics/process_ppi80_closeness.csv", sep=',', header=0, index_col=0)
ppi80_betweenness = pd.read_csv("../../data/processed/metrics/process_ppi80_betweenness.csv", sep=',', header=0, index_col=0)
ppi80_rwr = pd.read_csv("../../data/processed/metrics/process_ppi80_rwr.csv", sep=',', header=0, index_col=0)

In [6]:
importlib.reload(classifiers)
hypergeometric_simple_clf = classifiers.simple_classifier(reactome_hypergeometric, reactome_labels_df)

  0%|          | 0/433 [00:00<?, ?it/s]

In [7]:
hypergeometric_simple_clf.to_csv('../../models/process_hypergeometric_simple.csv', index=False)

In [18]:
importlib.reload(classifiers)
closeness_simple_clf = classifiers.simple_classifier(reactome_closeness, reactome_labels_df)

  0%|          | 0/433 [00:00<?, ?it/s]

In [19]:
closeness_simple_clf.to_csv('../../models/process_closeness_simple.csv', index=False)

In [10]:
importlib.reload(classifiers)
betweenness_simple_clf = classifiers.simple_classifier(reactome_betweenness, reactome_labels_df)

  0%|          | 0/433 [00:00<?, ?it/s]

In [11]:
betweenness_simple_clf.to_csv('../../models/process_betweenness_simple.csv', index=False)

In [84]:
importlib.reload(classifiers)
rwr_simple_clf = classifiers.simple_classifier(reactome_rwr, reactome_labels_df)

  0%|          | 0/433 [00:00<?, ?it/s]

In [104]:
rwr_simple_clf.to_csv('../../models/process_rwr_simple.csv', index=False)

In [22]:
ppi80_hyper_list = []
ppi80_closeness_list = []
ppi80_betweenness_list = []
ppi80_rwr_list = []
label_list = []

for i, df in ppi80_hyper.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_hyper_list.append(df)
    label_list.append(reactome_labels_df)

for i, df in ppi80_closeness.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_closeness_list.append(df)
    
for i, df in ppi80_betweenness.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_betweenness_list.append(df)
    
for i, df in ppi80_rwr.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_rwr_list.append(df)

In [106]:
importlib.reload(classifiers)
with concurrent.futures.ProcessPoolExecutor() as executor:
    ppi80_hyper_results = executor.map(classifiers.simple_classifier, ppi80_hyper_list, label_list)
ppi80_hyper_results_df = pd.concat(ppi80_hyper_results)
ppi80_hyper_results_df.to_csv('../../models/process_ppi80_hyper_simple.csv', index=False)
ppi80_hyper_results_df

,process,TP,FP,TN,FN,precision,recall,accuracy,f_measure,mcc,precision-recall area
0,R-HSA-1031716,52.0,9007.0,12222.0,21.0,0.005740,0.712329,0.576190,0.011389,0.034050,0.007167
1,R-HSA-112379,46.0,7057.0,14187.0,12.0,0.006476,0.793103,0.668153,0.012847,0.050945,0.004840
2,R-HSA-112385,46.0,7057.0,14187.0,12.0,0.006476,0.793103,0.668153,0.012847,0.050945,0.004840
3,R-HSA-1168640,45.0,3882.0,17369.0,6.0,0.011459,0.882353,0.817482,0.022624,0.088182,0.007713
4,R-HSA-1214188,4.0,688.0,20555.0,55.0,0.005780,0.067797,0.965121,0.010652,0.010497,0.002879
...,...,...,...,...,...,...,...,...,...,...,...
428,R-HSA-983147,173.0,10729.0,10358.0,42.0,0.015869,0.804651,0.494367,0.031124,0.059161,0.015475
429,R-HSA-983156,206.0,10644.0,10405.0,47.0,0.018986,0.814229,0.498122,0.037107,0.066864,0.018539
430,R-HSA-983157,206.0,10644.0,10405.0,47.0,0.018986,0.814229,0.498122,0.037107,0.066864,0.018539
431,R-HSA-983259,4.0,197.0,21046.0,55.0,0.019900,0.067797,0.988170,0.030769,0.031813,0.005454


In [23]:
importlib.reload(classifiers)
with concurrent.futures.ProcessPoolExecutor() as executor:
    ppi80_closeness_results = executor.map(classifiers.simple_classifier, ppi80_closeness_list, label_list)
ppi80_closeness_results_df = pd.concat(ppi80_closeness_results)
ppi80_closeness_results_df.to_csv('../../models/process_ppi80_closeness_simple.csv', index=False)
ppi80_closeness_results_df

,process,TP,FP,TN,FN,precision,recall,accuracy,f_measure,mcc,precision-recall area
0,R-HSA-1031716,3.0,3.0,21226.0,70.0,0.500000,0.041096,0.996573,0.075949,0.142628,0.054503
1,R-HSA-112379,26.0,4.0,21240.0,32.0,0.866667,0.448276,0.998310,0.590909,0.622630,0.544259
2,R-HSA-112385,26.0,4.0,21240.0,32.0,0.866667,0.448276,0.998310,0.590909,0.622630,0.544259
3,R-HSA-1168640,40.0,34.0,21217.0,11.0,0.540541,0.784314,0.997888,0.640000,0.650141,0.466130
4,R-HSA-1214188,14.0,135.0,21108.0,45.0,0.093960,0.237288,0.991550,0.134615,0.145627,0.031353
...,...,...,...,...,...,...,...,...,...,...,...
428,R-HSA-983147,151.0,5020.0,16067.0,64.0,0.029201,0.702326,0.761337,0.056071,0.108237,0.046960
429,R-HSA-983156,77.0,1011.0,20038.0,176.0,0.070772,0.304348,0.944278,0.114840,0.126128,0.069214
430,R-HSA-983157,77.0,1011.0,20038.0,176.0,0.070772,0.304348,0.944278,0.114840,0.126128,0.069214
431,R-HSA-983259,29.0,1967.0,19276.0,30.0,0.014529,0.491525,0.906253,0.028224,0.071946,0.011441


In [108]:
importlib.reload(classifiers)
with concurrent.futures.ProcessPoolExecutor() as executor:
    ppi80_betweenness_results = executor.map(classifiers.simple_classifier, ppi80_betweenness_list, label_list)
ppi80_betweenness_results_df = pd.concat(ppi80_betweenness_results)
ppi80_betweenness_results_df.to_csv('../../models/process_ppi80_betweenness.csv', index=False)
ppi80_betweenness_results_df

,process,TP,FP,TN,FN,precision,recall,accuracy,f_measure,mcc,precision-recall area
0,R-HSA-1031716,73.0,94.0,21135.0,0.0,0.437126,1.000000,0.995587,0.608333,0.659689,0.334242
1,R-HSA-112379,58.0,16.0,21228.0,0.0,0.783784,1.000000,0.999249,0.878788,0.884982,0.859620
2,R-HSA-112385,58.0,16.0,21228.0,0.0,0.783784,1.000000,0.999249,0.878788,0.884982,0.859620
3,R-HSA-1168640,51.0,31.0,21220.0,0.0,0.621951,1.000000,0.998545,0.766917,0.788063,0.468476
4,R-HSA-1214188,56.0,70.0,21173.0,3.0,0.444444,0.949153,0.996573,0.605405,0.648265,0.290713
...,...,...,...,...,...,...,...,...,...,...,...
428,R-HSA-983147,215.0,566.0,20521.0,0.0,0.275288,1.000000,0.973430,0.431727,0.517590,0.198628
429,R-HSA-983156,253.0,653.0,20396.0,0.0,0.279249,1.000000,0.969346,0.436583,0.520179,0.258627
430,R-HSA-983157,253.0,653.0,20396.0,0.0,0.279249,1.000000,0.969346,0.436583,0.520179,0.258627
431,R-HSA-983259,59.0,95.0,21148.0,0.0,0.383117,1.000000,0.995540,0.553991,0.617579,0.268560


In [109]:
importlib.reload(classifiers)
with concurrent.futures.ProcessPoolExecutor() as executor:
    ppi80_rwr_results = executor.map(classifiers.simple_classifier, ppi80_rwr_list, label_list)
ppi80_rwr_results_df = pd.concat(ppi80_rwr_results)
ppi80_rwr_results_df.to_csv('../../models/process_ppi80_rwr.csv', index=False)
ppi80_rwr_results_df

,process,TP,FP,TN,FN,precision,recall,accuracy,f_measure,mcc,precision-recall area
0,R-HSA-1031716,73.0,2.0,21227.0,0.0,0.973333,1.000000,0.999906,0.986486,0.986530,0.914575
1,R-HSA-112379,58.0,1.0,21243.0,0.0,0.983051,1.000000,0.999953,0.991453,0.991466,0.965210
2,R-HSA-112385,58.0,1.0,21243.0,0.0,0.983051,1.000000,0.999953,0.991453,0.991466,0.965210
3,R-HSA-1168640,51.0,0.0,21251.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.980392
4,R-HSA-1214188,56.0,6.0,21237.0,3.0,0.903226,0.949153,0.999578,0.925620,0.925694,0.724640
...,...,...,...,...,...,...,...,...,...,...,...
428,R-HSA-983147,215.0,32.0,21055.0,0.0,0.870445,1.000000,0.998498,0.930736,0.932268,0.801039
429,R-HSA-983156,253.0,57.0,20992.0,0.0,0.816129,1.000000,0.997324,0.898757,0.902175,0.744283
430,R-HSA-983157,253.0,57.0,20992.0,0.0,0.816129,1.000000,0.997324,0.898757,0.902175,0.744283
431,R-HSA-983259,59.0,1.0,21242.0,0.0,0.983333,1.000000,0.999953,0.991597,0.991608,0.937629
